# インストール

In [1]:
!pip install dtws1

# インポート

In [2]:
from dtws1 import heatmap, colorutil
from h3 import h3
import pandas as pd

/home/chen/work/venv/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/chen/work/venv/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/chen/work/venv/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful o

# データ準備

虎ノ門ヒルズの周辺のH3メッシュ

In [3]:
hills_lat, hills_lng = 35.66687568, 139.74947495
ids = list(h3.k_ring(h3.geo_to_h3(hills_lat, hills_lng, 10), 10))
print(ids[:3])

['8a2f5aade71ffff', '8a2f5aadea87fff', '8a2f5aadea2ffff']


H3メッシュの中心点の緯度・経度

In [4]:
lats = [h3.h3_to_geo(i)[0] for i in ids]
lngs = [h3.h3_to_geo(i)[1] for i in ids]
print(lats[0], lngs[0])

35.67549939304789 139.75629895657949


データフレームにまとめる 

In [5]:
df = pd.DataFrame(dict(
    id = ids,
    lat = lats,
    lng = lngs
))
df.head()

,id,lat,lng
0,8a2f5aade71ffff,35.675499,139.756299
1,8a2f5aadea87fff,35.665325,139.754344
2,8a2f5aadea2ffff,35.661308,139.752760
3,8a2f5aad8497fff,35.674406,139.741471
4,8a2f5aad8487fff,35.673423,139.740741


# 例

## 等間隔の境界値

`colorutil.color_selector`は、valuesで与えたリスト（的なオブジェクト）の最大値と最小値の間を`n`等分した値を色の境界にする

In [6]:
cs1 = colorutil.color_selector(values=df.lat, n=3) 
fmap1 = heatmap.draw_folium(
    df=df, 
    id_col="id",
    val_col="lat", 
    color_selector=cs1, 
    zoom_start=15
)
fmap1

## 要素数を揃える

`color_selector_p`は、各階級の要素数が、だいたい同じになるように境界値を選ぶ

In [7]:
cs2 = colorutil.color_selector_p(values=df.lat, n=3) 
fmap2 = heatmap.draw_folium(
    df=df, 
    id_col="id",
    val_col="lat", 
    color_selector=cs2, 
    zoom_start=15
)
fmap2

## 明示的に境界値を与える

`color_selector_tick`には、階級の境界値を与えることができる

In [8]:
cs3 = colorutil.color_selector_tick([35.6722, 35.6763])
fmap3 = heatmap.draw_folium(
    df=df, 
    id_col="id",
    val_col="lat", 
    color_selector=cs3, 
    zoom_start=15
)
fmap3

## 自分で色を定義する

値 -> 色(RGB16進数) という関数を自分で定義して与えることも可能

In [9]:
def cs4(v):
    if v > 35.670040:
        return "#ff0000"    
    elif v > 35.666023:
        return "#00ff00"
    else:
        return "#0000ff"   
fmap4 = heatmap.draw_folium(
    df=df, 
    id_col="id",
    val_col="lat", 
    color_selector=cs4, 
    zoom_start=15
)
fmap4

## その他
- 複数の列を`color_selector`の引数にすることはできない
  - あらかじめ、複数の列の情報をまとめた列を作っておく
- `hue`で、色味を変えることができる
  - 0〜255の値を与える
- `zoom_start`で、最初に表示される地図の倍率を変更できる
  - 0〜20の値を与える。数字が大きいほど倍率が大きい

In [10]:
from math import sqrt
df["dist"] = ((df.lat-hills_lat) ** 2 + (df.lng-hills_lng) ** 2).map(sqrt)
cs5 = colorutil.color_selector(df.dist, 10, hue=100)
fmap5 = heatmap.draw_folium(
    df=df, 
    id_col="id",
    val_col="dist", 
    color_selector=cs5, 
    zoom_start=13
)
fmap5

## 保存

In [11]:
fmap5.save("fmap5.html")

[fmap5.html](fmap5.html)